In [103]:
%run analizar.ipynb

In [104]:
# === Ejemplo de uso ===
codigo_fuente ="""
int suma(int a, int b){
  return a + b;
}
"""

In [105]:
# Analisis léxico
tokens = identificar(codigo_fuente)
print('Tokens encontrados: ')
for tipo, valor in tokens:
  print(f'{tipo}: {valor}')

Tokens encontrados: 
KEYWORD: int
IDENTIFIER: suma
DELIMITER: (
KEYWORD: int
IDENTIFIER: a
DELIMITER: ,
KEYWORD: int
IDENTIFIER: b
DELIMITER: )
DELIMITER: {
KEYWORD: return
IDENTIFIER: a
OPERATOR: +
IDENTIFIER: b
DELIMITER: ;
DELIMITER: }


In [ ]:
class Parser:
  def __init__(self, tokens):
    self.tokens = tokens
    self.pos = 0 #pos es de posicipon

  def obtener_token_actual(self):
    return self.tokens[self.pos] if self.pos < len(self.tokens) else None

  def coincidir(self, tipo_esperado):
    token_actual = self.obtener_token_actual()
    if token_actual and token_actual[0] == tipo_esperado: #Solo para verificar que haya algo, y no sea None
      self.pos += 1
      return token_actual
    else:
      raise SystemError(f'Error sintactico: se esperaba {tipo_esperado}, pero se encontro: {token_actual}')

  def parsear(self):
    #  Punto de entrada se espera una función
    self.funcion()
  
  def funcion(self):
    #  Gramatica para una funcion: int IDENTIFIER (int IIDENTIFIER) (cuerpo)
    self.coincidir('KEYWORD')  # Tipo de retorno (e). int()
    self.coincidir('IDENTIFIER')  # Nombre de la funcion
    self.coincidir('DELIMITER')  # Se espera un parentesis
    self.parametros()
    self.coincidir('DELIMITER')  # Se espera un parentesis de cierre ")"
    self.coincidir('DELIMITER')  # Se espera una llave de apertura "{"
    self.cuerpo()   # Se espera el cuerpo de la funcion
    self.coincidir('DELIMITER')  # Se espera una llave de cierre "}"
    
  def parametros(self):
    # Reglas para parametros: int IDENTIFIER (,int IDENTIFIER)*
    self.coincidir('KEYWORD')  # Tipo del parametro
    self.coincidir('IDENTIFIER')  # Nombre del parametro
    while self.obtener_token_actual() and self.obtener_token_actual()[1] == ',':
      self.coincidir('DELIMITER')  # Se espera una coma ","
      self.coincidir('KEYWORD')  # Tipo del parametro (int)
      self.coincidir('IDENTIFIER')  # Nombre del parametro

  def cuerpo(self):
    while self.obtener_token_actual() and self.obtener_token_actual()[1] != '}':
        if self.obtener_token_actual()[1] == 'return':
            self.declaracion()
        elif self.obtener_token_actual()[1] == 'while':
            self.ciclo_while()
        elif self.obtener_token_actual()[1] == 'if':
            self.condicional_if()
        elif self.obtener_token_actual()[1] == 'else':
            self.condicional_else()
        elif self.obtener_token_actual()[1] == 'for':
            self.ciclo_for()
        elif self.obtener_token_actual()[1] == 'print':
            self.imprimir()
        elif self.obtener_token_actual()[0] == 'IDENTIFIER':  # Si es un identificador
            siguiente = self.tokens[self.pos + 1] if self.pos + 1 < len(self.tokens) else None
            if siguiente and siguiente[0] == 'OPERATOR' and siguiente[1] in ('++', '--'):
                self.incremento()  # Detecta i++ o i--
                self.coincidir('DELIMITER')  # Punto y coma ";"
            else:
                self.asignacion()
        else:
            raise SyntaxError(f"Error sintáctico: instrucción inesperada {self.obtener_token_actual()}")

  def asignacion(self):
    self.coincidir('KEYWORD')  # Tipo de variable (ej. int)
    self.coincidir('IDENTIFIER')  # Nombre de la variable
    self.coincidir('OPERATOR')  # Operador de asignación "="
    self.expresion()  
    self.coincidir('DELIMITER')  # Punto y coma ";"

  def declaracion(self):
    self.coincidir('KEYWORD')  # return
    self.expresion()
    self.coincidir('DELIMITER')  # Punto y coma ";"

  def expresion(self):
    self.termino()
    while self.obtener_token_actual() and self.obtener_token_actual()[1] in ('+', '-', '==', '!=', '<', '>', '<=', '>=', '&&', '||'):
      self.coincidir('OPERATOR')
      self.termino()

  def termino(self):
    self.factor()
    while self.obtener_token_actual() and self.obtener_token_actual()[1] in ('*', '/'):
      self.coincidir('OPERATOR')
      self.factor()

  def factor(self):
    token_actual = self.obtener_token_actual()
    if token_actual[0] == 'NUMBER' or token_actual[0] == 'IDENTIFIER':
      self.coincidir(token_actual[0])
    elif token_actual[1] == '(':
      self.coincidir('DELIMITER')  # "("
      self.expresion()
      self.coincidir('DELIMITER')  # ")"
    else:
      raise SyntaxError(f'Error sintáctico en factor: token inesperado {token_actual}')

  def incremento(self):
    self.coincidir('IDENTIFIER')  # Nombre de la variable
    operador = self.obtener_token_actual()
    if operador and operador[0] == 'OPERATOR' and operador[1] in ('++', '--'):
        self.coincidir('OPERATOR')  # Coincide con ++ o --
    else:
        raise SyntaxError(f'Error sintáctico: se esperaba "++" o "--", pero se encontró "{operador}"')
      
  def instruccion_unica(self):
    token_actual = self.obtener_token_actual()
    if not token_actual:
        raise SyntaxError("Error sintáctico: se esperaba una instrucción")

    if token_actual[0] == "KEYWORD":
        if token_actual[1] == "return":
            self.declaracion()
        elif token_actual[1] == "while":
            self.ciclo_while()
        elif token_actual[1] == "if":
            self.condicional_if()
        elif token_actual[1] == "else":
            self.condicional_else()
        elif token_actual[1] == "for":
            self.ciclo_for()
        elif token_actual[1] == "print":
            self.imprimir()
        else:
            self.asignacion()
    elif token_actual[0] == "IDENTIFIER":
        self.asignacion()
    else:
        raise SyntaxError(f"Error sintáctico: instrucción inesperada {token_actual}")

  def ciclo_while(self):
    self.coincidir('KEYWORD')  # while
    self.coincidir('DELIMITER')  # "("
    self.expresion()  # Condición del ciclo
    self.coincidir('DELIMITER')  # ")"

    # Verificar si el siguiente token es una llave de apertura "{"
    if self.obtener_token_actual() and self.obtener_token_actual()[1] == "{":
        self.coincidir('DELIMITER')  # "{"
        self.cuerpo()  # Cuerpo del while
        self.coincidir('DELIMITER')  # "}"
    else:
        # Si no hay llaves, solo ejecutamos una única instrucción
        self.instruccion_unica()

  def ciclo_for(self):
    self.coincidir('KEYWORD')  # for
    self.coincidir('DELIMITER')  # "("
    if self.obtener_token_actual()[0] == 'KEYWORD':
        self.asignacion()  # Manejo de int i = 0;
    else:
        self.coincidir('IDENTIFIER')  # Permitir i = 0;
        self.coincidir('OPERATOR')  # =
        self.expresion()
        self.coincidir('DELIMITER')  # ";"
    self.expresion()  # Condición
    self.coincidir('DELIMITER')  # ";"
    self.incremento()  # Incremento
    self.coincidir('DELIMITER')  # ")"
    
    if self.obtener_token_actual() and self.obtener_token_actual()[1] == "{":
        self.coincidir('DELIMITER')  # "{"
        self.cuerpo()  # Cuerpo del for
        self.coincidir('DELIMITER')  # "}"
    else:
        self.instruccion_unica()

  def condicional_if(self):
    self.coincidir('KEYWORD')  # if
    self.coincidir('DELIMITER')  # "("
    self.expresion()  # Condición del if
    self.coincidir('DELIMITER')  # ")"

    if self.obtener_token_actual() and self.obtener_token_actual()[1] == "{":
        self.coincidir('DELIMITER')  # "{"
        self.cuerpo()  # Cuerpo del if
        self.coincidir('DELIMITER')  # "}"
    else:
        self.instruccion_unica()

  def condicional_else(self):
    self.coincidir('KEYWORD')  # else

    if self.obtener_token_actual() and self.obtener_token_actual()[1] == "{":
        self.coincidir('DELIMITER')  # "{"
        self.cuerpo()  # Cuerpo del else
        self.coincidir('DELIMITER')  # "}"
    else:
        self.instruccion_unica()


  def imprimir(self):
    self.coincidir('KEYWORD')  # print
    self.coincidir('DELIMITER')  # "("
    token_actual = self.obtener_token_actual()
    if token_actual[0] == 'STRING':
        self.coincidir('STRING')  # Cadena a imprimir
    else:
      self.expresion()  # Expresion a imprimir
  
    self.coincidir('DELIMITER')  # ")"
    self.coincidir('DELIMITER')  # ";"
  
  # PARA EFECTOS DEL EJERCICIO NO SE IMPLEMENTARON LAS SIGUIENTES FUNCIONES DEL CODIGO ORIGINAL 
  '''
  def cuerpo(self):
      # Reglas para el cuerpo de la funcion: return IDENTIFIER OPERATOR IDENTIFIER;
      self.coincidir('KEYWORD')  #  Se espera una palabra reservada return
      self.coincidir('IDENTIFIER')  # Se espera el nombre de la variable
      self.coincidir('OPERATOR')  # Se espera un operador "+"
      self.coincidir('IDENTIFIER')  # Se espera el nombre de la variable
      self.coincidir('DELIMITER')  # Se espera un punto y coma ";"              
  '''

## EJERCICIO 10/02/2025

In [ ]:
texto_prueba = """
int suma(int a, int b) {
    for (int i = 0; i < 10; i++) {
        if (i == 5){
            while (i < 10) {
                print(i);
                if (i == 8)
                    print("i es igual a 8");
                i++;
            }
        }    
    }
}

"""

In [ ]:
tokens_ejercicio = identificar(texto_prueba)

In [ ]:
try:
    parser = Parser(tokens_ejercicio)
    parser.parsear()
    print('Analisis sintactico exitoso')
except SyntaxError as e:
    print(e)

Analisis sintactico exitoso


## Arbol de Sintaxis

In [ ]:
class nodoAST:
    # Clase base para todos los nodos del AST
    pass
    
class NodoFuncion(nodoAST):
    def __init__(self, nombre, parametros, cuerpo):
        self.nombre = nombre
        self.parametros = parametros
        self.cuerpo = cuerpo

In [111]:
class NodoParametro(nodoAST):
    # Nodo que representa un parametro de función
    def __init__(self, tipo, nombre):
        self.tipo = tipo
        self.nombre = nombre

In [112]:
class NodoAsignacion(nodoAST):
    # Nodo representa una asiganción de variable
    def __init__(self, nombre, expresion):
        self.nombre = nombre
        self.expresion = expresion

In [113]:
class NodoOperacion(nodoAST):
    # Nodo que representa una operacion aritmetica
    def __init__(self, izquierda, operador, derecha):
        self.izquierda = izquierda
        self.operador = operador
        self.derecha = derecha

In [114]:
class NodoRetorno(nodoAST):
    # Nodo que representa a a sentencia return
    def __init__(self, expresion):
        self.expresion = expresion

In [115]:
class NodoIdentificador(nodoAST):
    def __init__(self, nombre):
        self.nombre = nombre
        
class NodoNumero(nodoAST):
    # Nodo que representa un número
    def __init__(self, valor):
        self.valor = valor